# EDF Exploration


In [1]:
import os
import sys
from pathlib import Path
import time
import tempfile
import logging
import dateutil.parser
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mne
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, LocalCluster

In [2]:
# packageroot = Path('../../').resolve()
packageroot = Path('../..').resolve()
print(packageroot)
sys.path.append(str(packageroot))

from pythoneeg import core
from pythoneeg import visualization
from pythoneeg import constants

print(core.__all__)
# print(visualization.__all__)
# print(dir(constants))

/mnt/isilon/marsh_single_unit/PythonEEG


/mnt/isilon/marsh_single_unit/PythonEEG/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['DDFBinaryMetadata', 'LongRecordingOrganizer', 'convert_ddfcolbin_to_ddfrowbin', 'convert_ddfrowbin_to_si', 'convert_units_to_multiplier', 'convert_colpath_to_rowpath', 'filepath_to_index', 'is_day', 'set_temp_directory', 'get_temp_directory', 'parse_path_to_animalday', 'parse_str_to_genotype', 'parse_str_to_animal', 'parse_str_to_day', 'parse_chname_to_abbrev', 'nanaverage', 'LongRecordingAnalyzer', 'MountainSortAnalyzer', 'FragmentAnalyzer', '_log_transform']


In [3]:
core.set_temp_directory('/scr1/users/dongjp')

## Testing Sox5 Data

In [4]:
# cluster_general = SLURMCluster(
#         cores=8,
#         memory='20GB',
#         walltime='12:00:00',
#         interface=None,
#         scheduler_options={'interface': 'eth1'},
#         job_extra_directives=['--output=/dev/null',
#                              '--error=/dev/null']
#     )
# cluster_general.scale(jobs=10)

# cluster_spike = SLURMCluster(
#         cores=1,
#         memory='10GB',
#         processes=1,
#         walltime='12:00:00',
#         interface=None,
#         scheduler_options={'interface': 'eth1'},
#         job_extra_directives=['--output=/dev/null',
#                              '--error=/dev/null']
#     )
# cluster_spike.scale(12)

# cluster_general.wait_for_workers(10)
# cluster_spike.wait_for_workers(12)

In [5]:
# Set up logger with detailed formatting
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.DEBUG, stream=sys.stdout, force=True)
logger = logging.getLogger()


In [6]:
# import spikeinterface.extractors as se
# from neo.io import EDFIO
# from pyedflib import highlevel
import mne

edfpath = Path('/mnt/isilon/marsh_single_unit/PythonEEG Data/Arx Parv/29 32 34/')
temppath = edfpath / 'MARSH 20110606-PARV-29-32-34 2011-06-07 16H03M_2_Selection1.EDF'
temppath = str(temppath)


data = mne.io.read_raw_edf(temppath)
# mne.concatenate_raws()

Extracting EDF parameters from /mnt/isilon/marsh_single_unit/PythonEEG Data/Arx Parv/29 32 34/MARSH 20110606-PARV-29-32-34 2011-06-07 16H03M_2_Selection1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [7]:
display(data.info)
display(data.duration.item() / 60)
display(data.ch_names)

<Info | 8 non-empty values
 bads: []
 ch_names: EEG E1-REF1, EEG E2-REF1, EEG E3-REF1, EEG E4-REF1, EEG E5-REF1, ...
 chs: 32 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2011-06-09 15:55:20 UTC
 nchan: 32
 projs: []
 sfreq: 2000.0 Hz
 subject_info: <subject_info | his_id: 20110606-Parv-29-32-34, sex: 1, first_name: Marsh, last_name: 20110606-Parv-29-32-34, birthday: 2001-01-01>
>

240.033

['EEG E1-REF1',
 'EEG E2-REF1',
 'EEG E3-REF1',
 'EEG E4-REF1',
 'EEG E5-REF1',
 'EEG E6-REF1',
 'EEG E7-REF1',
 'EEG E8-REF1',
 'EEG E9-REF2',
 'EEG E10-REF2',
 'EEG E11-REF2',
 'EEG E12-REF2',
 'EEG E13-REF2',
 'EEG E14-REF2',
 'EEG E15-REF2',
 'EEG E16-REF2',
 'EEG E17-REF3',
 'EEG E18-REF3',
 'EEG E19-REF3',
 'EEG E20-REF3',
 'EEG E21-REF3',
 'EEG E22-REF3',
 'EEG E23-REF3',
 'EEG E24-REF3',
 'EEG E25-REF4',
 'EEG E26-REF4',
 'EEG E27-REF4',
 'EEG E28-REF4',
 'EEG E29-REF4',
 'EEG E30-REF4',
 'EEG E31-REF4',
 'EEG E32-REF4']

In [8]:
# %matplotlib qt
# data.plot(duration=10, scalings='auto')

In [9]:
data_folders_to_animal_ids = {}

In [10]:
lro = core.LongRecordingOrganizer(edfpath)
lro.convert_file_with_mne_to_recording(extract_func=mne.io.read_raw_edf, mode='files', file_pattern='*.EDF')

Extracting EDF parameters from /mnt/isilon/marsh_single_unit/PythonEEG Data/Arx Parv/29 32 34/MARSH 20110606-PARV-29-32-34 2011-06-07 16H03M_1_Selection1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


Extracting EDF parameters from /mnt/isilon/marsh_single_unit/PythonEEG Data/Arx Parv/29 32 34/MARSH 20110606-PARV-29-32-34 2011-06-07 16H03M_1_Selection2.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/isilon/marsh_single_unit/PythonEEG Data/Arx Parv/29 32 34/MARSH 20110606-PARV-29-32-34 2011-06-07 16H03M_1_Selection3.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/isilon/marsh_single_unit/PythonEEG Data/Arx Parv/29 32 34/MARSH 20110606-PARV-29-32-34 2011-06-07 16H03M_1_Selection4.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/isilon/marsh_single_unit/PythonEEG Data/Arx Parv/29 32 34/MARSH 20110606-PARV-29-32-34 2011-06-07 16H03M_1_Selection5.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameter

MemoryError: Unable to allocate 109. GiB for an array with shape (32, 457269840) and data type float64

In [9]:
wars = []
for data_folder, animal_ids in data_folders_to_animal_ids.items():
    print(data_folder)
    for animal_id in animal_ids:
        print(f'\t{animal_id}')
        import warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            ao = visualization.AnimalOrganizer(data_parent_folder / data_folder, animal_id,
                                    mode="nest", 
                                    assume_from_number=True,
                                    skip_days=['bad', 'Empty'])
        
        # with Client(cluster_general) as client:
        #     client.upload_file(str(packageroot / 'pythoneeg.zip'))
        #     display(client)
            
        #     ao.convert_colbins_to_rowbins(overwrite=True, multiprocess_mode='dask')
        #     ao.convert_rowbins_to_rec(multiprocess_mode='dask') # paralleization breaks if not enough memory
        #     war = ao.compute_windowed_analysis(['all'], multiprocess_mode='dask')

        # with Client(cluster_spike) as client:
        #     client.upload_file(str(packageroot / 'pythoneeg.zip'))
        #     display(client)
            
        #     sars = ao.compute_spike_analysis(multiprocess_mode='dask')
        #     for sar in sars:
        #         sar.save_fif_and_json(Path(f'./test-sars-full/{sar.animal_day}-{sar.bin_folder_name}').resolve(), overwrite=True)
        #     war.read_sars_spikes(sars)
            
        # war.save_pickle_and_json(Path(f'./test-wars-full/{animal_id}').resolve())
        # wars.append(war)

010822_cohort4_group2_2mice_MWT_MHET
	M3
2025-04-22 21:42:12,365 - INFO - bin_folder_pattern: /mnt/isilon/marsh_single_unit/PythonEEG Data Bins/Sox5/Dr. Lefebvre Project/010822_cohort4_group2_2mice_MWT_MHET/*M3*/*
2025-04-22 21:42:12,366 - INFO - self._bin_folders: ['/mnt/isilon/marsh_single_unit/PythonEEG Data Bins/Sox5/Dr. Lefebvre Project/010822_cohort4_group2_2mice_MWT_MHET/M3_MHET_cohort4_group2/010822_cohort4_group2_M3_MHET_files0-12', '/mnt/isilon/marsh_single_unit/PythonEEG Data Bins/Sox5/Dr. Lefebvre Project/010822_cohort4_group2_2mice_MWT_MHET/M3_MHET_cohort4_group2/010922_cohort4_group2_M3_MHET_files22-39', '/mnt/isilon/marsh_single_unit/PythonEEG Data Bins/Sox5/Dr. Lefebvre Project/010822_cohort4_group2_2mice_MWT_MHET/M3_MHET_cohort4_group2/011022_cohort4_group2_M3_MHET_files87-97', '/mnt/isilon/marsh_single_unit/PythonEEG Data Bins/Sox5/Dr. Lefebvre Project/010822_cohort4_group2_2mice_MWT_MHET/M3_MHET_cohort4_group2/011022_cohort4_group2_M3_MHET_files53-65', '/mnt/isilon/m

2025-04-22 21:42:12,475 - DEBUG - Before prune: 13 colbins, 13 rowbins, 13 metas
2025-04-22 21:42:12,499 - DEBUG - After prune: 13 colbins, 13 rowbins, 13 metas
2025-04-22 21:42:12,795 - DEBUG - Before prune: 18 colbins, 18 rowbins, 18 metas
2025-04-22 21:42:12,797 - DEBUG - After prune: 18 colbins, 18 rowbins, 18 metas
2025-04-22 21:42:12,958 - DEBUG - Before prune: 11 colbins, 11 rowbins, 11 metas
2025-04-22 21:42:12,959 - DEBUG - Removing 01082022_cohort4_2mice_MHET-96
2025-04-22 21:42:12,960 - DEBUG - After prune: 10 colbins, 10 rowbins, 10 metas
2025-04-22 21:42:13,094 - DEBUG - Before prune: 13 colbins, 13 rowbins, 13 metas
2025-04-22 21:42:13,096 - DEBUG - After prune: 13 colbins, 13 rowbins, 13 metas
2025-04-22 21:42:13,288 - DEBUG - Before prune: 21 colbins, 21 rowbins, 21 metas
2025-04-22 21:42:13,290 - DEBUG - After prune: 21 colbins, 21 rowbins, 21 metas
2025-04-22 21:42:13,471 - DEBUG - Before prune: 13 colbins, 13 rowbins, 13 metas
2025-04-22 21:42:13,472 - DEBUG - After 